In [1]:
import findspark

In [2]:
findspark.init("/opt/manual/spark")

In [3]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import *

In [4]:
!ls -l $SPARK_HOME/jars | grep postgresql-42.2.14.jar

-rw-rw-r--. 1 train train   932808 Jul 22  2020 postgresql-42.2.14.jar


In [5]:
spark = (
    SparkSession.builder
    .appName("JDBC and SQL")
    .master("yarn")
    .config("sparks.jars.packages", "org.apache.spark:spark-avro_2.12:3.0.0")
    .enableHiveSupport()         
.getOrCreate())

In [7]:
df = spark.read \
.format("csv") \
.option("header",True) \
.option("inferSchema", True) \
.option("compression", "gzip") \
.csv("/user/train/datasets/Hotel_Reviews.csv.gz")

In [10]:
df2 = df.withColumn("Tags",
                    F.split(F.col("Tags"), ",")
                    .cast(ArrayType(StringType()))) \
.withColumn("Review_Date", F.to_date(F.col("Review_Date"), "M/d/yyyy"))

In [11]:
df2.printSchema()

root
 |-- Hotel_Address: string (nullable = true)
 |-- Additional_Number_of_Scoring: integer (nullable = true)
 |-- Review_Date: date (nullable = true)
 |-- Average_Score: double (nullable = true)
 |-- Hotel_Name: string (nullable = true)
 |-- Reviewer_Nationality: string (nullable = true)
 |-- Negative_Review: string (nullable = true)
 |-- Review_Total_Negative_Word_Counts: integer (nullable = true)
 |-- Total_Number_of_Reviews: integer (nullable = true)
 |-- Positive_Review: string (nullable = true)
 |-- Review_Total_Positive_Word_Counts: integer (nullable = true)
 |-- Total_Number_of_Reviews_Reviewer_Has_Given: integer (nullable = true)
 |-- Reviewer_Score: double (nullable = true)
 |-- Tags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- days_since_review: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- lng: string (nullable = true)



# Write to PostreSQL

In [12]:
jdbcUrl = "jdbc:postgresql://localhost/traindb?user=train&password=Ankara06"

In [13]:
df2.write \
.jdbc(url=jdbcUrl, table="hotel_reviews", mode="overwrite", properties={"driver": "org.postgresql.Driver"})

In [14]:
spark.stop()

In [ ]:
df.select(col("date"), )